# Projeto CyberDefense: Deteção de Intrusão
**Analista:** [Seu Nome]

## 1. Contexto
O objetivo é classificar conexões de rede como **Normal** ou **Intrusão**.
Temos de lidar com dados duplicados provenientes de um erro no servidor de logs.

In [ ]:
# ==========================================
# Importação de Bibliotecas
# ==========================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Dataset e Pré-processamento
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Modelos
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB

# Métricas
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, classification_report

# Configuração de estilo visual
sns.set(style="whitegrid")
print("Bibliotecas carregadas com sucesso!")

In [ ]:
# =========================================
# GERAÇÃO DE DADOS SINTÉTICOS DE REDE
# Vamos criar 1000 registos de tráfego
# =========================================
X, y = make_classification(
    n_samples=1000,
    n_features=5,
    n_informative=3,
    n_redundant=0,
    n_classes=2,
    weights=[0.8, 0.2], # 20% são ataques (Intrusão), 80% normal
    random_state=42
)

# Criar DataFrame com nomes realistas de Cibersegurança
df = pd.DataFrame(X, columns=['Packet_Size', 'Latency_ms', 'Login_Attempts', 'Port_Number', 'Encrypted_Bits'])
df['is_intrusion'] = y # 0 = Normal, 1 = Intrusão

# SIMULAR DUPLICATAS (ERRO DE LOGS)
# Duplicar as primeiras 100 linhas e misturar
duplicatas = df.iloc[:100]
df_sujo = pd.concat([df, duplicatas], ignore_index=True)

print(f"Dados gerados! Dimensão do dataset: {df_sujo.shape}")
df_sujo.head()

In [ ]:
# ==========================================
# Deteção de Anomalias de Dados
# ==========================================

print(f"Dimensão com duplicatas (Dataset Sujo): {df_sujo.shape}")
print(f"Número de registos duplicados: {df_sujo.duplicated().sum()}")

In [ ]:
# ==========================================
# Limpeza de Dados (Remoção de Redundância)
# ==========================================

# Como discutido na aula: Duplicatas no treino/teste criam um viés otimista irrealista.
# Solução: Deduplicação

df_limpo = df_sujo.drop_duplicates()

print(f"Dimensão após limpeza: {df_limpo.shape}")
print(f"Verificação de duplicatas restantes: {df_limpo.duplicated().sum()}")

# Separação de X (Atributos) e y (Target)
X = df_limpo.drop('is_intrusion', axis=1)
y = df_limpo['is_intrusion']

In [ ]:
# ==========================================
# Divisão Treino/Teste e Normalização
# ==========================================

# 1. Divisão Hold-out (70% Treino, 30% Teste)
# 'stratify=y' garante que mantemos a proporção de intrusão/normal em ambos os conjuntos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# 2. Normalização (StandardScaler)
# IMPORTANTE: O KNN e a Regressão Logística são sensíveis à escala dos dados.
# O Naive Bayes (Gaussiano) lida bem sem, mas mal não faz neste contexto comparativo.
scaler = StandardScaler()

# Ajustamos (fit) apenas ao TREINO para evitar vazamento de dados (data leakage)
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Dados divididos e normalizados.")
print(f"Treino: {X_train_scaled.shape[0]} amostras")
print(f"Teste: {X_test_scaled.shape[0]} amostras")

In [ ]:
# ==========================================
# Treino dos Modelos
# ==========================================

# Dicionário para armazenar os nossos classificadores
modelos = {
    "KNN (k=3)": KNeighborsClassifier(n_neighbors=3),
    "Regressão Logística": LogisticRegression(random_state=42),
    "Naive Bayes": GaussianNB()
}

resultados = {}

print("A treinar modelos...")
for nome, modelo in modelos.items():
    # Treinar
    modelo.fit(X_train_scaled, y_train)
    # Prever
    y_pred = modelo.predict(X_test_scaled)
    # Guardar previsões para avaliação
    resultados[nome] = y_pred
    print(f"-> {nome} treinado.")

In [ ]:
# ==========================================
# Avaliação e Comparação Visual
# ==========================================

# Função auxiliar para desenhar Matriz de Confusão
def plot_conf_matrix(y_true, y_pred, title):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(5, 4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False,
                xticklabels=['Normal (0)', 'Intrusão (1)'],
                yticklabels=['Normal (0)', 'Intrusão (1)'])
    plt.xlabel('Previsto')
    plt.ylabel('Real')
    plt.title(f'Matriz de Confusão: {title}')
    plt.show()

# Lista para guardar métricas finais
comparativo = []

for nome, y_pred in resultados.items():
    # 1. Calcular Métricas
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred)
    rec = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    # Adicionar à lista comparativa
    comparativo.append({
        "Modelo": nome,
        "Accuracy": acc,
        "Precision": prec,
        "Recall": rec,
        "F1-Score": f1
    })

    # 2. Mostrar Matriz de Confusão
    plot_conf_matrix(y_test, y_pred, nome)

# Criar DataFrame final de comparação
df_comparativo = pd.DataFrame(comparativo).set_index("Modelo")

In [ ]:
# ==========================================
# CÉLULA 7: Tabela Final de Resultados
# ==========================================

print("--- COMPARAÇÃO DE PERFORMANCE ---")
# Formatar para mostrar percentagens
display(df_comparativo.style.format("{:.2%}").background_gradient(cmap='Greens'))

# Análise rápida (print):
print("\nObservações para a Aula:")
print("1. Accuracy: Qual modelo acertou mais no geral?")
print("2. Recall: Qual modelo foi melhor a detetar a intrusão (Classe 1)? (Evitou Falsos Negativos)")
print("3. Naive Bayes: Foi competitivo mesmo assumindo independência dos atributos?")